In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
output_dir = '/content/drive/MyDrive/02 data/text_ads_generation'

In [5]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models
from google.colab import userdata
import yaml


gcp_project = userdata.get('GCP_PROJECT')
vertexai.init(project=gcp_project, location="us-central1")

def generate(prompt, model_id="gemini-1.0-pro-001", max_output_tokens=2048, temperature=0.7, top_p=1, return_yaml=True, tries=3):
  model = GenerativeModel(model_id)

  for _ in range(tries):
    try:
      response = model.generate_content(
        prompt,
        generation_config={
            "max_output_tokens": max_output_tokens,
            "temperature": temperature,
            "top_p": top_p
        },
        safety_settings={
            generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        },
        stream=False,
      )

      if return_yaml:
        text = response.text
        if text.startswith('```yaml'):
          text = text[7:-3]
        if text.endswith('```'):
          text = text[:-3]
        return yaml.safe_load(text)
      else:
        return response.text
    except Exception as e:
      print(f'Error in generation {response}', e)

  return None

In [6]:
prompt = """I\'m planning to build an e-commerce website like Amazon. Do the following for me:
1. Generate 20 top level categories of products that I can sell
2. Generate a list of 5-10 sub categories for each top level categories.
Return the result in yaml format."""
print(generate(prompt, return_yaml=True, tries=1))

{'top_level_categories': ['Electronics - Televisions - Smartphones - Laptops - Tablets - Cameras - Smartwatches - Speakers - Headphones - Gaming consoles - Virtual reality headsets', "Clothing - Men's clothing - Shirts - Pants - Suits - Jackets - Shoes - Accessories - Women's clothing - Dresses - Tops - Bottoms - Skirts - Shoes - Accessories - Children's clothing - Boys' clothing - Girls' clothing - Baby clothing - Shoes - Men's shoes - Women's shoes - Children's shoes - Accessories - Jewelry - Watches - Bags - Sunglasses - Hats", 'Home & Kitchen - Furniture - Sofas - Chairs - Tables - Beds - Storage - Kitchen appliances - Refrigerators - Stoves - Ovens - Microwaves - Dishwashers - Home décor - Curtains - Rugs - Pillows - Artwork - Plants - Kitchenware - Pots and pans - Cookware - Bakeware - Utensils - Gadgets - Cleaning supplies - Vacuum cleaners - Mops - Brooms - Cleaning solutions - Paper products', 'Health & Beauty - Skincare - Cleansers - Moisturizers - Serums - Masks - Sunscreen 

In this notebook, we will build a dataset for text ads generation.

Currently suportted customer types:
- E-Commerce
- Game
- Apps

Let's build them one by one.

# E-Commerce

## Generate categories

Prompt used:
```
I'm planning to build an e-commerce website like Amazon. Do the following for me:

1. Generate 20 top level categories of products that I can sell
2. Generate a list of sub categories for each top level categories.

Return the result in yaml format.
```

In [7]:
%%writefile e_commerce_categories.yaml
Electronics:
  - Computers & Accessories
  - Mobile Phones & Accessories
  - TV & Home Theater
  - Cameras & Optics
  - Wearable Technology
  - Audio Equipment

Fashion:
  - Women's Clothing
  - Men's Clothing
  - Kids' Clothing
  - Shoes
  - Watches
  - Jewelry & Accessories

Home & Kitchen:
  - Furniture
  - Home Decor
  - Kitchen & Dining
  - Bedding & Bath
  - Appliances
  - Patio & Garden

Beauty & Personal Care:
  - Makeup
  - Skin Care
  - Hair Care
  - Fragrance
  - Grooming & Hygiene
  - Oral Care

Sports & Outdoors:
  - Exercise & Fitness
  - Outdoor Recreation
  - Team Sports
  - Cycling
  - Camping & Hiking
  - Water Sports

Books:
  - Fiction
  - Non-Fiction
  - Children's Books
  - Textbooks
  - Magazines

Toys & Games:
  - Action Figures & Collectibles
  - Dolls & Accessories
  - Puzzles
  - Board Games
  - Video Games
  - Outdoor Play

Automotive:
  - Car Parts & Accessories
  - Motorcycle Parts & Accessories
  - Tires & Wheels
  - Tools & Equipment
  - Automotive Care

Tools & Home Improvement:
  - Power Tools
  - Hand Tools
  - Hardware
  - Plumbing
  - Electrical
  - Painting Supplies

Pet Supplies:
  - Dog Supplies
  - Cat Supplies
  - Fish & Aquatic Pets
  - Bird Supplies
  - Small Animal Supplies
  - Horse Supplies

Baby:
  - Baby Clothing
  - Diapering
  - Feeding
  - Baby Gear
  - Nursery
  - Toys & Books

Office Products:
  - Office Furniture
  - Office Electronics
  - Printing & Writing
  - Stationery & Supplies
  - Janitorial & Safety

Grocery & Gourmet Food:
  - Pantry Staples
  - Baking & Cooking
  - Snacks & Beverages
  - International Foods
  - Organic & Specialty Foods
  - Meal Solutions

Health & Household:
  - Vitamins & Supplements
  - Household Supplies
  - Medical Supplies
  - Cleaning Supplies
  - Air & Water Treatment
  - Vision Care

Industrial & Scientific:
  - Lab & Scientific
  - Industrial Supplies
  - Janitorial & Sanitation
  - Material Handling
  - Power Transmission
  - Safety & Security

Handmade:
  - Jewelry & Accessories
  - Home Decor
  - Art & Sculpture
  - Clothing & Shoes
  - Bags & Cases
  - Baby & Kids

Arts, Crafts & Sewing:
  - Crafting
  - Painting & Drawing
  - Sewing
  - Knitting & Crochet
  - Scrapbooking
  - Stamping & Embossing

Musical Instruments:
  - Guitars & Basses
  - Keyboards & Pianos
  - Drums & Percussion
  - DJ & Lighting Equipment
  - Recording Equipment
  - Scores & Sheet Music

Movies, Music & Games:
  - Movies & TV Shows
  - Music
  - Video Games
  - Software
  - Digital Comics
  - Digital Magazines

Writing e_commerce_categories.yaml


Now, for each sub category, generate X different products and descriptions.

In [ ]:
import yaml
import json
from tqdm.notebook import tqdm

num_products_per_category = 20

def iterate_categories(config_file):
  with open(config_file, 'r') as f:
    config = yaml.safe_load(f)

  for category, sub_categories in config.items():
      for sub_category in sub_categories:
          yield f'{category} -> {sub_category}'


def normalize_headline(h):
  if h.startswith('\"') and h.endswith('\"'):
    return h[1:-1]
  elif h.startswith('\'') and h.endswith('\''):
    return h[1:-1]
  return h


def generate_ads(product, count=10):
  prompt = '\n'.join([
      f"I'm running an online store selling products. Write {count}",
      "text ad headlines for my product using the information below.",
      yaml.dump(product),
      "Return your answers in yaml format.",
      "Each item should contain a single key called `headline`."
      "Quote the all the headlines in dobule quotes."
      "Each headline should be as short as possible, about 3-5 words."
  ])

  results = []

  for _ in range(3):
    ad_headlines = generate(prompt, return_yaml=True, tries=3)
    if not ad_headlines:
      continue
    for item in ad_headlines:
      if 'headline' in item:
        h = normalize_headline(item['headline'])
        if len(h) <= 30:
          results.append(h)
        if len(results) >= count:
          return results

  return results

categories = list(iterate_categories('e_commerce_categories.yaml'))

with open(f'{output_dir}/e_commerce.jsonl', 'w') as f:
  for cat in tqdm(categories):
    tqdm.write(f'Generating for category: {cat}')

    prompt = '\n'.join([
      "I'm running an online store selling products. Do the following task:",
      f"1. Generate {num_products_per_category} product names in the category: {cat}",
      "2. Generate target audience for each product.",
      "3. Generate a product description for each product.",
      "4. Generate selling points for each product.",
      "Return your answers in yaml format.",
      "Each product item should contain keys called `product_name`, `target_audience`, `description` and `selling_point`."
    ])

    products = generate(prompt, return_yaml=True, tries=5)

    if not products:
      print(f'Failed to generate products for {cat}, skip.')
      continue

    for product in tqdm(products):
      try:
        product['product_category'] = cat
        ad_headlines = generate_ads(product)
        product['ad_headlines'] = ad_headlines
        json.dump(product, f)
        f.write('\n')
      except:
        print(f'Failed to generate ads for {product}, skip.')



# Apps

In [13]:
%%writefile app_categories.yaml
categories:
  - name: Games
    subcategories:
      - Action
      - Adventure
      - Puzzle
      - Simulation
      - Role-Playing (RPG)
      - Sports
      - Strategy

  - name: Social Media
    subcategories:
      - Social Networking
      - Blogging & Microblogging
      - Dating
      - Forums & Communities
      - Video Sharing

  - name: Productivity
    subcategories:
      - To-Do Lists
      - Notes & Writing
      - Office Suites
      - Calendar & Scheduling
      - Time Tracking
      - Finance & Budgeting

  - name: Entertainment
    subcategories:
      - Music Streaming
      - Video Streaming
      - E-books & Reading
      - Podcasts
      - News & Magazines

  - name: Lifestyle
    subcategories:
      - Shopping & E-commerce
      - Food & Drink
      - Travel
      - Home & Design
      - Fashion

  - name: Photo & Video
    subcategories:
      - Image Editing
      - Video Editing
      - Camera Tools
      - Social Sharing (Photo/Video Focused)

  - name: Health & Fitness
    subcategories:
      - Workout & Exercise
      - Nutrition & Diet
      - Sleep Tracking
      - Meditation & Mindfulness
      - Health Monitoring

  - name: Education
    subcategories:
      - Learning Languages
      - Educational Games
      - Online Courses
      - Reference & Study Tools
      - Kids Education

  - name: Navigation & Maps
    subcategories:
      - GPS Navigation
      - Public Transportation
      - Location Sharing
      - Travel Guides

  - name: Utilities
    subcategories:
      - File Management
      - Weather
      - Calculators
      - Password Managers
      - Device Customization

  - name: Communication
    subcategories:
      - Email
      - Messaging
      - Video Calls
      - Voice Calls (VoIP)

  - name: Business
    subcategories:
      - Project Management
      - Collaboration
      - CRM (Customer Relationship Management)
      - Cloud Storage
      - Remote Work Tools

  - name: Creativity
    subcategories:
      - Drawing & Painting
      - Music Production
      - 3D Modeling
      - Graphic Design

  - name: Security
    subcategories:
      - Antivirus & Antimalware
      - VPN (Virtual Private Network)
      - Parental Controls
      - Identity Protection

  - name: Kids
    subcategories:
      - Games (Age-Appropriate)
      - Learning & Development
      - Creativity Tools
      - Parental Controls

  - name: News & Information
    subcategories:
        - News Aggregators
        - General News
        - Specialized News (Tech, Sports, Finance, etc.)
        - Weather

  - name: Accessibility
    subcategories:
      - Screen Readers
      - Text Magnification
      - Speech-to-Text
      - Assistive Touch Features

  - name: Augmented Reality (AR)
    subcategories:
      - AR Games
      - Shopping & Visualization
      - AR Navigation
      - Educational AR

  - name: Personalization
    subcategories:
      - Wallpapers & Themes
      - Ringtones
      - Launchers
      - Widgets

  - name: Finance
    subcategories:
      - Banking
      - Investing
      - Budgeting
      - Cryptocurrency
      - Expense Tracking



Writing app_categories.yaml


In [19]:
import yaml
import json
from tqdm.notebook import tqdm

num_products_per_category = 20

def iterate_categories(config_file):
  with open(config_file, 'r') as f:
    config = yaml.safe_load(f)

  for category in config['categories']:
      for sub_category in category['subcategories']:
          yield f'{category["name"]} -> {sub_category}'


def normalize_headline(h):
  if h.startswith('\"') and h.endswith('\"'):
    return h[1:-1]
  elif h.startswith('\'') and h.endswith('\''):
    return h[1:-1]
  return h


def generate_ads(product, count=10):
  prompt = '\n'.join([
      f"I'm building a mobile app. Write {count}",
      "text ad headlines for my product using the information below.",
      yaml.dump(product),
      "Return your answers in yaml format.",
      "Quote the headline with double quotes if necessary.",
      "Each item should contain a single key called `headline`.",
      "Quote the all the headlines in dobule quotes.",
      "Each headline should be as short as possible, about 3-5 words.",
  ])

  results = []

  for _ in range(3):
    ad_headlines = generate(prompt, return_yaml=True, tries=3)
    if not ad_headlines:
      continue
    for item in ad_headlines:
      if 'headline' in item:
        h = normalize_headline(item['headline'])
        if len(h) <= 30:
          results.append(h)
        if len(results) >= count:
          return results

  return results

categories = list(iterate_categories('app_categories.yaml'))

with open(f'{output_dir}/games.jsonl', 'w') as f:
  for cat in tqdm(categories):
    tqdm.write(f'Generating for category: {cat}')

    prompt = '\n'.join([
      "I'm brainstorming on some ideas of building a new mobile app. Do the following task:",
      f"1. Generate {num_products_per_category} app names in the category: {cat}",
      "2. Generate target audience for each app.",
      "3. Generate a description for each app.",
      "4. Generate selling points for each app.",
      "Quote the all the headlines in dobule quotes if necessary.",
      "Return your answers in yaml format.",
      "Each app item should contain keys called `app_name`, `target_audience`, `description` and `selling_point`."
    ])

    products = generate(prompt, return_yaml=True, tries=5)

    if not products:
      print(f'Failed to generate products for {cat}, skip.')
      continue

    for product in tqdm(products):
      try:
        product['product_category'] = cat
        ad_headlines = generate_ads(product)
        product['ad_headlines'] = ad_headlines
        json.dump(product, f)
        f.write('\n')
      except:
        print(f'Failed to generate ads for {product}, skip.')



  0%|          | 0/93 [00:00<?, ?it/s]

Generating for category: Games -> Action


  0%|          | 0/19 [00:00<?, ?it/s]

Error in generation candidates {
  content {
    role: "model"
    parts {
      text: "- headline: \"\"Embark on an Epic Adventure\"\"\n- headline: \"\"Explore a Vast Open World\"\"\n- headline: \"\"Uncover Ancient Secrets\"\"\n- headline: \"\"Battle Formidable Enemies\"\"\n- headline: \"\"Search for Your Lost Sibling\"\"\n- headline: \"\"Beautiful Graphics and Storyline\"\"\n- headline: \"\"Wide Cast of Characters\"\"\n- headline: \"\"Anime-Inspired Open-World RPG\"\"\n- headline: \"\"Action-Packed Adventure\"\"\n- headline: \"\"Unforgettable Gaming Experience\"\""
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

Generating for category: Finance -> Cryptocurrency


  0%|          | 0/20 [00:00<?, ?it/s]

Generating for category: Finance -> Expense Tracking


  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
!cat "{output_dir}/games.jsonl"|wc -l

1797
